# Data manipulation
* Container: Data Science (studio)

## 0. Install packages

In [1]:
install_needed = True  # should only be True once

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## 1. Get dataset

In [1]:
import pandas as pd

In [3]:
store_info = pd.read_csv("../dataset/emart_store_info.csv", header=0)
store_tenant_info = pd.read_csv("../dataset/emart_store_tenant_info.csv", header=0)

## 2. Preprocessing

### 2.1. Q and A generation

In [4]:
import boto3

## Korean

In [4]:
%%time
store_id_map, ques_ans, cnt = {}, {}, 0
seperator = "<sep>"

for row in store_info.itertuples():
    
    store_id = getattr(row, "id")
    name, zipcode, address1 = getattr(row, "name"), getattr(row, "zipcode"), getattr(row, "address1")
    address3, tel, shopping_time = getattr(row, "address3"), getattr(row, "tel"), getattr(row, "shopping_time")
    holiday_info, parking_count, intro = getattr(row, "holiday_info"), getattr(row, "parking_count"), getattr(row, "intro")
    map_info = getattr(row, "map_info")
    
    name_trans = name
            
    ques_ans[f'{name}의 우편번호는 무엇입니까?'] = f'{name}의 우편번호는 다음과 같습니다. \n{zipcode}' + seperator + name_trans
    ques_ans[f'{name}의 주소는 무엇입니까?'] = f'{name}의 주소는 다음과 같습니다. \n{address1}' + seperator + name_trans
    ques_ans[f'{name}의 전화번호는 무엇입니까?'] = f'{name}의 전화번호는 다음과 같습니다. \n{tel}' + seperator + name_trans
    
    if holiday_info != "[]":
        holiday_info = ', '.join(holiday_info[1:-1].split(","))
        ques_ans[f'{name}의 휴일은 언제입니까?'] = f'{name}의 휴일은 다음과 같습니다/. \n{holiday_info}' + seperator + name_trans
        
    #ques_ans[f'{name}의 최대 몇대까지 주차 가능합니까?'] = f'{name}의 주차가능수는 다음과 같습니다. \n{parking_count}대' + seperator + name_trans
    #ques_ans[f'{name}의 주차정보를 알려주세요'] = f'{name}의 주차정보는 다음과 같습니다. \n{intro}'+ seperator + name_trans 
    
    parking = f'{name}의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \n{intro}\n 매장 전체 주차가능대수: {parking_count}대'
    parking_char_count = len(parking)
    
    if parking_char_count >= 450:
        
        sep_token_index = int(len(parking.split("\n"))/2)
        
        parking_1 = "".join(parking.split("\n")[:sep_token_index])
        parking_2 = f'{name}의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \n' + "".join(parking.split("\n")[sep_token_index:])
        
        ques_ans[f'{name}의 주차정보를 알려주세요-1'] = parking_1 +seperator + name_trans
        ques_ans[f'{name}의 주차정보를 알려주세요-2'] = parking_2 +seperator + name_trans

    else:
        ques_ans[f'{name}의 주차정보를 알려주세요'] = f'{name}의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \n{intro}\n 전체 매장 주차가능 수: {parking_count}대' +seperator + name_trans
        
    if str(map_info) != "nan": ques_ans[f'{name}의 오시는 길을 알려주세요'] = f'{name}의 오시는 길은 다음과 같습니다. \n{map_info}' + seperator + name_trans
        
    store_id_map[store_id] = name
    
    cnt += 1
    if cnt % 1000 == 0: print (cnt)
    #if cnt == 10: break

for row in store_tenant_info.itertuples():
    
    jijum_id, category_name, store_name = getattr(row, "jijum_id"), getattr(row, "category_name"), getattr(row, "store_name")
    tel, store_shopping_time = getattr(row, "tel"), getattr(row, "store_shopping_time")
    
    name = store_id_map[jijum_id]
    question, answer = f'{name}에 {store_name}가 있습니까?', f'네, {name}에는 {store_name}가 있고, 전화번호는 {tel}, 운영시간은 {store_shopping_time} 입니다.'
    
    cnt += 1
    if cnt % 1000 == 0: print (cnt)
    
    
    ques_ans[question] = answer + seperator + name

1000
2000
3000
4000
5000
6000
CPU times: user 18.5 ms, sys: 7.82 ms, total: 26.3 ms
Wall time: 24.7 ms


In [5]:
for key in ques_ans.keys():
    if "주차정보를 알려주세요" in key:
        print (key, ques_ans[key])
    

이마트 창동점의 주차정보를 알려주세요-1 이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. &lt;유료주차 운영 안내&gt;- 24시간운영- 무료회차 시간 30분이내- 기본요금: 10분당 1,000원- 30분 초과시 회차시간 포함 요금징수※ 구매금액별 무료주차시간    1만원 이상 구매 2시간 무료주차    3만원 이상 구매 3시간 무료주차    5만원 이상 구매 4시간 무료주차   10만원 이상 구매 5시간 무료주차<sep>이마트 창동점
이마트 창동점의 주차정보를 알려주세요-2 이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. 
※ 임대매장(스타벅스, 식당 등)이용 무료주차    해당매장에서 무료주차 2시간 적용※ 계산대 출력영수증, 모바일영수증에     하단 바코드를 사용하여    신용카드 전용 사전무인정산기 정산후 출차    -케이엠파크:080-330-3600※ 전기차 충전소 위치: 옥외 주차장 8F     차지비,완속 3대,24:00~24:00, 고객센터 1600-4047 매장 전체 주차가능대수: 159대<sep>이마트 창동점
이마트 분당점의 주차정보를 알려주세요-1 이마트 분당점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. 사전 / 무인 정산 유료 주차장 (신용카드 전용)시행 일자 : 2019년 8월 16일(목)※ 주차 요금   무료회차 30분이내   기본요금 10분당 1,000원   주차장 운영시간 09:30~21:30 -일최대 요금 20,000원입니다. / 30분초과시   회차시간 포함 요금 징수 됩니다. ※구매 금액별 무료 주차 시간    (일 최대 5시간 무료)   1만원 이상 2시간    3만원 이상 3시간    5만원 이상 4시간<sep>이마트 분당점
이마트 분당점의 주차정보를 알려주세요-2 이마트 분당점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. 
  10만원 이상 5시간  [유의사항] 1. 이마트 계산대에서 발행된 당일 영수증만 사용가능 합니다.  2. 그 외 영수증은

### 2.2. save dataset

In [6]:
import pickle

In [7]:
def to_pickle(obj, save_path):
        
    with open(file=save_path, mode="wb") as f:
        pickle.dump(obj, f)

def from_pickle(obj_path):

    with open(file=obj_path, mode="rb") as f:
        obj=pickle.load(f)

    return obj

In [8]:
to_pickle(ques_ans, "../dataset/quenstion_answer_ko.pkl")

In [9]:
ques_ans = from_pickle("../dataset/quenstion_answer_ko.pkl")

In [10]:
ques_ans

{'이마트 창동점의 우편번호는 무엇입니까?': '이마트 창동점의 우편번호는 다음과 같습니다. \n132045<sep>이마트 창동점',
 '이마트 창동점의 주소는 무엇입니까?': '이마트 창동점의 주소는 다음과 같습니다. \n서울 도봉구 노해로 65길 4<sep>이마트 창동점',
 '이마트 창동점의 전화번호는 무엇입니까?': '이마트 창동점의 전화번호는 다음과 같습니다. \n02-901-1234<sep>이마트 창동점',
 '이마트 창동점의 휴일은 언제입니까?': '이마트 창동점의 휴일은 다음과 같습니다/. \n2023-07-09, 2023-07-23, 2023-06-11, 2023-06-25, 2023-01-22<sep>이마트 창동점',
 '이마트 창동점의 주차정보를 알려주세요-1': '이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. &lt;유료주차 운영 안내&gt;- 24시간운영- 무료회차 시간 30분이내- 기본요금: 10분당 1,000원- 30분 초과시 회차시간 포함 요금징수※ 구매금액별 무료주차시간    1만원 이상 구매 2시간 무료주차    3만원 이상 구매 3시간 무료주차    5만원 이상 구매 4시간 무료주차   10만원 이상 구매 5시간 무료주차<sep>이마트 창동점',
 '이마트 창동점의 주차정보를 알려주세요-2': '이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. \n※ 임대매장(스타벅스, 식당 등)이용 무료주차    해당매장에서 무료주차 2시간 적용※ 계산대 출력영수증, 모바일영수증에     하단 바코드를 사용하여    신용카드 전용 사전무인정산기 정산후 출차    -케이엠파크:080-330-3600※ 전기차 충전소 위치: 옥외 주차장 8F     차지비,완속 3대,24:00~24:00, 고객센터 1600-4047 매장 전체 주차가능대수: 159대<sep>이마트 창동점',
 '이마트 창동점의 오시는 길을 알려주세요': '이마트 창동점의 오시는 길은 다음과 같습니다. \n[지하철 이용시] \n창동역 

## 3. To csv

In [11]:
import pandas as pd

In [12]:
ques_ans_df = pd.DataFrame(ques_ans.items(), columns=['Category', 'Information'])
ques_ans_df["Source"] = ques_ans_df['Information'].str.split('<sep>', expand=True)[1]
ques_ans_df[["Information", "Source"]].to_csv("../dataset/quenstion_answer_ko.csv", index=False)

In [60]:
ques_ans_df

,Category,Information,Source
0,이마트 창동점의 우편번호는 무엇입니까?,이마트 창동점의 우편번호는 다음과 같습니다. \n132045<sep>이마트 창동점,이마트 창동점
1,이마트 창동점의 주소는 무엇입니까?,이마트 창동점의 주소는 다음과 같습니다. \n서울 도봉구 노해로 65길 4<sep>...,이마트 창동점
2,이마트 창동점의 전화번호는 무엇입니까?,이마트 창동점의 전화번호는 다음과 같습니다. \n02-901-1234<sep>이마트...,이마트 창동점
3,이마트 창동점의 휴일은 언제입니까?,"이마트 창동점의 휴일은 다음과 같습니다/. \n2023-07-09, 2023-07-...",이마트 창동점
4,이마트 창동점의 주차정보를 알려주세요-1,"이마트 창동점의 주차정보, 주차요금, 무료주차 정보는 다음과 같습니다. &lt;유료...",이마트 창동점
...,...,...,...
8477,트레이더스 홀세일 클럽 하남점에 베스킨라빈스(아이스크림)가 있습니까?,"네, 트레이더스 홀세일 클럽 하남점에는 베스킨라빈스(아이스크림)가 있고, 전화번호는...",트레이더스 홀세일 클럽 하남점
8478,트레이더스 홀세일 클럽 하남점에 SK통신사(SK모바일/SK브로드밴드)가 있습니까?,"네, 트레이더스 홀세일 클럽 하남점에는 SK통신사(SK모바일/SK브로드밴드)가 있고...",트레이더스 홀세일 클럽 하남점
8479,이마트 의왕점에 전기차충전소(에스트레픽) 급속 2대가 있습니까?,"네, 이마트 의왕점에는 전기차충전소(에스트레픽) 급속 2대가 있고, 전화번호는 -3...",이마트 의왕점
8480,트레이더스 홀세일 클럽 안성점에 베이커리가 있습니까?,"네, 트레이더스 홀세일 클럽 안성점에는 베이커리가 있고, 전화번호는 031-8092...",트레이더스 홀세일 클럽 안성점


In [14]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

model_id, model_version, = (
    "huggingface-textgeneration1-gpt-j-6b",
    "*",
)

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.

deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

In [15]:
deploy_image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117'